Book Reference: page 191-207 of **Python for Data Analysis Book by Wes McKinney**

# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up **80% or more of an analyst’s time**

## Handling Missing Data
- For numeric data, pandas uses the floating-point value NaN (Not a number) to represent missing data. (This is called _sentinel value_ that can be easily detected.
- In statistics applications NA data may either be
    1. data that does not exist
    2. data that exists but was not observed through problems with data collection, for example
- When cleaning up data for analysis, it is often important to do **analysis on the missing data itself** to identify data collection problems or **potential biases** in the data caused by missing data.

In [6]:
import pandas as pd
import numpy as np

In [7]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [9]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [10]:
string_data[0] = None

In [12]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

In [13]:
from numpy import nan as NA

In [15]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()
# On a Series, 
# dropna() returns the Series with only the non-null data and index values:

0    1.0
2    3.5
4    7.0
dtype: float64

In [16]:
# data.dropna() is equivalent to
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

dropna by default drops any row containing a missing value in a DataFrame

In [17]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
     [NA, NA, NA], [NA, 6.5, 3.]])

In [18]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
cleaned = data.dropna()

In [21]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA:

In [22]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1:`

In [23]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [25]:
data.dropna(axis=1, how='all') # column 4 is dropped

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the `thresh` argument:

In [26]:
df = pd.DataFrame(np.random.randn(7, 3))

In [28]:
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

In [29]:
df

,0,1,2
0,0.873934,NaN,NaN
1,-0.799741,NaN,NaN
2,-0.321652,NaN,0.141925
3,0.470914,NaN,0.770180
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


In [30]:
df.dropna()

,0,1,2
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


In [31]:
df

,0,1,2
0,0.873934,NaN,NaN
1,-0.799741,NaN,NaN
2,-0.321652,NaN,0.141925
3,0.470914,NaN,0.770180
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


In [32]:
df.dropna()

,0,1,2
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


In [33]:
df.dropna(thresh=2)

,0,1,2
2,-0.321652,NaN,0.141925
3,0.470914,NaN,0.770180
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


### Filling in Missing Data

In [34]:
df.fillna(0)

,0,1,2
0,0.873934,0.000000,0.000000
1,-0.799741,0.000000,0.000000
2,-0.321652,0.000000,0.141925
3,0.470914,0.000000,0.770180
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


In [36]:
df.fillna({1: 0.5, 2: 0}) # different fill value for each column

,0,1,2
0,0.873934,0.500000,0.000000
1,-0.799741,0.500000,0.000000
2,-0.321652,0.500000,0.141925
3,0.470914,0.500000,0.770180
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


`fillna` returns a new object, but you can modify the existing object in-place

In [39]:
_ = df.fillna(0, inplace=True)

In [40]:
df

,0,1,2
0,0.873934,0.000000,0.000000
1,-0.799741,0.000000,0.000000
2,-0.321652,0.000000,0.141925
3,0.470914,0.000000,0.770180
4,0.764263,-0.592170,-0.759425
5,0.876531,1.565893,-0.239873
6,0.728873,0.056431,0.886560


In [41]:
# The same interpolation methods available for reindexing 
# can be used with fillna:
# ffill forward fill
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

In [42]:
df

,0,1,2
0,-0.721525,-1.195323,1.449691
1,-0.274775,0.203900,-1.896144
2,0.504573,NaN,-0.364040
3,-0.406958,NaN,-0.655256
4,1.558481,NaN,NaN
5,-0.067167,NaN,NaN


In [43]:
df.fillna(method='ffill')

,0,1,2
0,-0.721525,-1.195323,1.449691
1,-0.274775,0.203900,-1.896144
2,0.504573,0.203900,-0.364040
3,-0.406958,0.203900,-0.655256
4,1.558481,0.203900,-0.655256
5,-0.067167,0.203900,-0.655256


In [44]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.721525,-1.195323,1.449691
1,-0.274775,0.203900,-1.896144
2,0.504573,0.203900,-0.364040
3,-0.406958,0.203900,-0.655256
4,1.558481,NaN,-0.655256
5,-0.067167,NaN,-0.655256


With fillna you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [47]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [48]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

In [49]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
     'k2': [1, 1, 2, 3, 3, 4, 4]})

In [50]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [51]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [52]:
# drop_duplicates returns a DataFrame where the duplicated array is False:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


---

In [54]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [55]:
data.drop_duplicates(['k1']) # filter duplicates only based on the 'k1' column

,k1,k2,v1
0,one,1,0
1,two,1,1


In [58]:
# duplicated and drop_duplicates by default keep the first observed value combination.
# as shown from prev example

#Passing keep='last' will return the last one:
data.drop_duplicates(['k1', 'k2'], keep='last') # row with index '5' is dropped

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping

In [59]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
     'Pastrami', 'corned beef', 'Bacon',
     'pastrami', 'honey ham', 'nova lox'],
     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [60]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food
came from. Let’s write down a mapping of each distinct meat type to the kind of
animal:

In [61]:
meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}

In [62]:
lowercased = data['food'].str.lower()

In [63]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [65]:
data['animal'] = lowercased.map(meat_to_animal) # map func

In [66]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [67]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.

### Replacing Values

In [69]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [70]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be **sentinel values** for missing data

In [71]:
data.replace(-999, np.nan) # use inplace = True if you want to replace the values inplace

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [72]:
# If you want to replace multiple values at once, 
# you instead pass a list and then the substitute value
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [73]:
# To use a different replacement for each value, pass a list of substitutes:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [75]:
# other method 
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [76]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four'])

In [77]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [79]:
# Like a Series, the axis indexes have a map method
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [80]:
data.index = data.index.map(transform)

In [89]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [90]:
# If you want to create a transformed version of a dataset 
# without modifying the original, a useful method is rename:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Indiana,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, `rename` can be used in conjunction with a dict-like object providing new values
for a subset of the axis labels:

In [91]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


---

In [95]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [98]:
data.rename(index={'Ohio': 'INDIANA'}, inplace=True)

In [99]:
data

,one,two,three,four
INDIANA,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets:

In [100]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [101]:
# Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, 
# and finally 61 and older. To
# do so, you have to use cut, a function in pandas:
bins = [18, 25, 35, 60, 100]

In [103]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [104]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [105]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [106]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

---

In [107]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

---

In [109]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

---

If you pass an integer number of bins to cut instead of explicit bin edges, it will compute
equal-length bins based on the minimum and maximum values in the data.
Consider the case of some uniformly distributed data chopped into fourths:

In [123]:
data = np.random.randint(20, size=20)
data

array([ 2, 12, 10,  8, 17,  4, 13, 15, 15, 17, 14, 10, 10, 14, 11,  5, 10,
        5, 12, 16])

In [125]:
pd.cut(data, 4, precision=0)

[(2.0, 6.0], (10.0, 13.0], (10.0, 13.0], (6.0, 10.0], (13.0, 17.0], ..., (2.0, 6.0], (10.0, 13.0], (2.0, 6.0], (10.0, 13.0], (13.0, 17.0]]
Length: 20
Categories (4, interval[float64]): [(2.0, 6.0] < (6.0, 10.0] < (10.0, 13.0] < (13.0, 17.0]]

---

In [126]:
data = np.random.randn(1000) # Normally distributed

In [129]:
cats = pd.qcut(data, 4) # roughly equal-size bins

In [130]:
cats

[(-0.669, -0.0411], (0.665, 3.478], (-0.669, -0.0411], (-0.669, -0.0411], (-0.0411, 0.665], ..., (0.665, 3.478], (-0.669, -0.0411], (-3.34, -0.669], (-0.669, -0.0411], (-0.669, -0.0411]]
Length: 1000
Categories (4, interval[float64]): [(-3.34, -0.669] < (-0.669, -0.0411] < (-0.0411, 0.665] < (0.665, 3.478]]

In [131]:
pd.value_counts(cats)

(0.665, 3.478]       250
(-0.0411, 0.665]     250
(-0.669, -0.0411]    250
(-3.34, -0.669]      250
dtype: int64

---

In [132]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) # Similar to cut you can pass your own quantiles

[(-1.363, -0.0411], (1.25, 3.478], (-1.363, -0.0411], (-1.363, -0.0411], (-0.0411, 1.25], ..., (1.25, 3.478], (-1.363, -0.0411], (-1.363, -0.0411], (-1.363, -0.0411], (-1.363, -0.0411]]
Length: 1000
Categories (4, interval[float64]): [(-3.34, -1.363] < (-1.363, -0.0411] < (-0.0411, 1.25] < (1.25, 3.478]]

### Detecting and Filtering Outliers

In [133]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067281,0.022500,-0.003188,0.003517
std,1.020652,1.032049,0.982751,1.024004
min,-3.477141,-3.156626,-2.818838,-3.115825
25%,-0.764080,-0.717213,-0.688984,-0.719314
50%,-0.087429,-0.002566,-0.029294,0.023214
75%,0.576792,0.711671,0.671458,0.712005
max,3.573715,3.229440,2.876007,2.930524


In [136]:
data.head()

,0,1,2,3
0,-0.199208,-1.025779,-1.111755,-0.660458
1,-0.972532,-1.139488,0.360461,1.221294
2,0.589166,-0.591603,-0.728287,-1.325999
3,-0.650042,-0.154849,-0.775889,-0.275178
4,0.847596,-0.822920,-2.614220,-0.872766


In [138]:
col = data[2] # get column 2
col.head()

0   -1.111755
1    0.360461
2   -0.728287
3   -0.775889
4   -2.614220
Name: 2, dtype: float64

Suppose you wanted to find values in one of the columns exceeding 3 in absolute
value:

In [139]:
col[np.abs(col) > 3]

Series([], Name: 2, dtype: float64)

In [145]:
# To select all rows having a value exceeding 3 or –3, 
# you can use the any method on a boolean DataFrame:
data[(np.abs(data) > 3).any(1)]  # any == Return whether any element is True over requested axis

,0,1,2,3
33,0.530198,-3.013361,0.740853,0.209296
194,0.717223,-3.086690,-0.841878,-0.280890
299,-3.098698,0.293645,1.160243,-0.957175
567,-1.402812,3.214463,0.731675,1.131312
598,-0.456641,3.229440,-0.064788,0.707724
662,-0.624791,-3.111444,-0.551427,1.041481
821,-0.389169,-3.156626,-0.416572,0.304975
843,0.549608,-0.814287,-1.867437,-3.115825
848,3.573715,0.928155,-0.452972,0.079475
960,-3.477141,1.477344,-0.769376,1.486342


In [150]:
# Values can be set based on these criteria. 
# Here is code to cap values outside the interval –3 to 3:
data[np.abs(data) > 3] = np.sign(data) * 3 
# np.sign returns sign of the num (if positive or negative) -1 or 1
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067279,0.022425,-0.003188,0.003633
std,1.016985,1.029616,0.982751,1.023658
min,-3.000000,-3.000000,-2.818838,-3.000000
25%,-0.764080,-0.717213,-0.688984,-0.719314
50%,-0.087429,-0.002566,-0.029294,0.023214
75%,0.576792,0.711671,0.671458,0.712005
max,3.000000,3.000000,2.876007,2.930524


In [151]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,1.0,1.0
2,1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,-1.0


### Permutation and Random Sampling
- Permutation / randomly reordering

In [153]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [155]:
sampler = np.random.permutation(5)
sampler

array([4, 3, 2, 1, 0])

That array can then be used in iloc-based indexing or the equivalent take function:

In [156]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [157]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3


To select a random subset without replacement, you can use the `sample` method on
Series and DataFrame:

In [158]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7
